In [1]:
!pip install torch-scatter torch-sparse torch-cluster torch-spline-conv torch-geometric -f https://data.pyg.org/whl/torch-1.12.0+cu116.html

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in links: https://data.pyg.org/whl/torch-1.12.0+cu116.html
     |████████████████████████████████| 8.0 MB 893 kB/s 
     |████████████████████████████████| 3.5 MB 668 kB/s 
     |████████████████████████████████| 2.4 MB 1.0 MB/s 
     |████████████████████████████████| 706 kB 882 kB/s 
     |████████████████████████████████| 407 kB 5.1 MB/s 
  Created wheel for torch-geometric: filename=torch_geometric-2.0.4-py3-none-any.whl size=616603 sha256=8f8309e32a96fd0a17472176c7d957ae834c618dbf32522c5f4f81b10c71ce8a
  Stored in directory: /root/.cache/pip/wheels/18/a6/a4/ca18c3051fcead866fe7b85700ee2240d883562a1bc70ce421
Successfully built torch-geometric


In [2]:
import torch
import torch.nn.functional as F

from torch_geometric.data import Data
from torch_geometric.datasets import Planetoid
from torch_geometric.nn import GCNConv


In [3]:
edge_idx = torch.tensor([
                         [0, 1], 
                         [1, 0], 
                         [1, 2],
                         [2, 1]
                        ], 
                        dtype=torch.long)

x = torch.tensor([[-1], [0], [1]], dtype=torch.float)

data = Data(x=x, edge_index=edge_idx.t().contiguous())
print(data)
print(f'data.x = 3 nodes each with 1 feature')
print(f'data.edge_index --> 4/2 = 2 undirected edges')
print(data.is_directed())
print(data.has_self_loops())

Data(x=[3, 1], edge_index=[2, 4])
data.x = 3 nodes each with 1 feature
data.edge_index --> 4/2 = 2 undirected edges
False
False


In [4]:
dataset = Planetoid(root='/tmp/Cora', name='Cora')
print(dataset)

Cora()


Processing...
Done!


In [11]:
class GCN(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = GCNConv(dataset.num_node_features, 16)
        self.conv2 = GCNConv(16, dataset.num_classes)

    def forward(self, data):
        x, edge_idx = data.x, data.edge_index
        
        x = self.conv1(x, edge_idx)
        x = F.relu(x)
        x = F.dropout(x, training=self.training)
        x = self.conv2(x, edge_idx)
        x = F.log_softmax(x, dim=1)

        return x


In [12]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = GCN().to(device)
data = dataset[0].to(device)

optim = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)

In [13]:
model.train()

for epoch in range(200):
    optim.zero_grad()
    out = model(data)

    loss = F.nll_loss(out[data.train_mask], data.y[data.train_mask])
    loss.backward()

    optim.step()

In [15]:
model.eval()

pred = model(data).argmax(dim=1)
correct = (pred[data.test_mask] == data.y[data.test_mask]).sum()

accuracy = int(correct) / int(data.test_mask.sum())

print(f'Accuracy: {accuracy:.4f}')

Accuracy: 0.7950
